# Before 

In [ ]:
# imports 
import numpy as np
import pandas as pd 
import random,os

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder,StandardScaler
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

TRAIN_PATH = "../input/spaceship-titanic/train.csv"
TEST_PATH = "../input/spaceship-titanic/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/spaceship-titanic/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "PassengerId"
TARGET = "Transported"
DELETE_COL = ["Name","Cabin"]
BOOL_COL = ["CryoSleep","VIP"]

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()

MODEL_MAX_DEPTH = 5
MODEL_TASK_TYPE = 'GPU'
MODEL_RL = 0.01
MODEL_EVAL_METRIC ='Accuracy'
MODEL_LOSS_FUNCTION = 'Logloss'
MODEL_ESR = 10
MODEL_VERBOSE = 100
MODEL_ITERATIONS = 2000

CROSS_VAL_CV = 5

# Preprocess

In [ ]:
# load 
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
    
train = train.drop(DELETE_COL,axis=1)
test = test.drop(DELETE_COL,axis=1)

# check null
def checkNull_fillData(df):
    for col in df.columns:
        if len(df.loc[df[col].isnull() == True]) != 0:
            if df[col].dtype == "float64" or df[col].dtype == "int64":
                df.loc[df[col].isnull() == True,col] = df[col].median()
            else:
                df.loc[df[col].isnull() == True,col] = df[col].mode()[0]
                
checkNull_fillData(train)
checkNull_fillData(test)

# object -> int
train[TARGET] = train[TARGET].astype(int)
for col in BOOL_COL:
    train[col] = train[col].astype(int)
    test[col] = test[col].astype(int)

# check duplicated data
feature = [col for col in train.columns if col != ID and col != TARGET]
train = train[train[feature].duplicated()==False]

#standard scaler 
num_col = []
for col in train.columns:
    if train[col].dtypes != "object" and col != TARGET and col != ID:
        num_col.append(col)
        
scaler = StandardScaler()
train[num_col] = scaler.fit_transform(train[num_col])
test[num_col] = scaler.transform(test[num_col])

# label encoding
str_col = []
for col in train.columns:
    if train[col].dtypes == "object" and col != TARGET and col != ID:
        str_col.append(col)

for col in str_col:
    encoder = LabelEncoder()
    encoder.fit(train[col])
    train[col] = encoder.transform(train[col])

    for label in np.unique(test[col]):
        if label not in encoder.classes_: 
            encoder.classes_ = np.append(encoder.classes_, label) 
    test[col] = encoder.transform(test[col])

# Build

In [ ]:
# import module we'll need to import our custom module
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/stratified-shuffle-split-oof-py/function_stratifiedshufflesplit_oof.py", dst = "../working/function_stratifiedshufflesplit_oof.py")

from function_stratifiedshufflesplit_oof import *

X = train.drop([ID,TARGET],axis =1)
y = train[TARGET]
X_test = test.drop([ID],axis=1)

model = CatBoostClassifier(
    verbose=MODEL_VERBOSE,
    early_stopping_rounds=MODEL_ESR,
    random_seed=SEED,
    max_depth=MODEL_MAX_DEPTH,
    task_type=MODEL_TASK_TYPE,
    learning_rate=MODEL_RL,
    iterations=MODEL_ITERATIONS,
    loss_function=MODEL_LOSS_FUNCTION,
    eval_metric= MODEL_EVAL_METRIC
)

preds = stratifiedShuffleSplitOOF(CROSS_VAL_CV,X,y,X_test,model,accuracy_score)

# After

In [ ]:
submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[TARGET] =preds.astype(bool)
submission.to_csv(SUBMISSION_PATH,index=False)
submission.head()